# Other sources

# Install dep

In [ ]:
!pip install --quiet transformers
!pip install --quiet pandas
!pip install --quiet numpy

In [ ]:
!python --version
!pip --version

Python 3.7.10
pip 19.3.1 from /usr/local/lib/python3.7/dist-packages/pip (python 3.7)


# Download data

In [ ]:
!mkdir data

In [ ]:
!wget https://raw.githubusercontent.com/susanli2016/NLP-with-Python/master/data/title_conference.csv -P data

In [ ]:
!ls -l data
!head -n 20 data/title_conference.csv

total 184
-rw-r--r-- 1 root root 188402 Jun 10 13:24 title_conference.csv
Title,Conference
Innovation in Database Management: Computer Science vs. Engineering.,VLDB
High performance prime field multiplication for GPU.,ISCAS
enchanted scissors: a scissor interface for support in cutting and interactive fabrication.,SIGGRAPH
Detection of channel degradation attack by Intermediary Node in Linear Networks.,INFOCOM
Pinning a Complex Network through the Betweenness Centrality Strategy.,ISCAS
Analysis and Design of Memoryless Interconnect Encoding Scheme.,ISCAS
Dynamic bluescreens.,SIGGRAPH
A Quantitative Assured Forwarding Service.,INFOCOM
Automatic sanitization of social network data to prevent inference attacks.,WWW
A &#916;&#931; IR-UWB radar with sub-mm ranging capability for human body monitoring systems.,ISCAS
Architecture of a multi-slot main memory system for 3.2 Gbps operation.,ISCAS
Rule-based Service Customization via Houdini.,WWW
Business Policy Modeling and Enforcement in Databa

# Import deps

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import random

# Set the env

In [ ]:
if torch.cuda.is_available():
   device = torch.device("cuda:0")
   print("Running on the GPU")
else:
   device = torch.device("cpu")
   print("Running on the CPU")

print(device)

Running on the GPU
cuda:0


In [ ]:
# seed random
seed_val = 17

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# global variables
epochs = 5

# Load data

In [ ]:
df = pd.read_csv('data/title_conference.csv')
df.head()

,Title,Conference
0,Innovation in Database Management: Computer Sc...,VLDB
1,High performance prime field multiplication fo...,ISCAS
2,enchanted scissors: a scissor interface for su...,SIGGRAPH
3,Detection of channel degradation attack by Int...,INFOCOM
4,Pinning a Complex Network through the Betweenn...,ISCAS


In [ ]:
d = df[ df['Conference'] == 'VLDB' ]
d.head()


In [ ]:
df['Conference'].value_counts()

ISCAS       864
INFOCOM     515
VLDB        423
WWW         379
SIGGRAPH    326
Name: Conference, dtype: int64

# Encoding the labels

In [ ]:
possible_labels = df['Conference'].unique()
print(possible_labels)

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index

print(label_dict)

['VLDB' 'ISCAS' 'SIGGRAPH' 'INFOCOM' 'WWW']


{'INFOCOM': 3, 'ISCAS': 1, 'SIGGRAPH': 2, 'VLDB': 0, 'WWW': 4}

In [ ]:
df['label'] = df.Conference.replace(label_dict)
df.head()

,Title,Conference,label
0,Innovation in Database Management: Computer Sc...,VLDB,0
1,High performance prime field multiplication fo...,ISCAS,1
2,enchanted scissors: a scissor interface for su...,SIGGRAPH,2
3,Detection of channel degradation attack by Int...,INFOCOM,3
4,Pinning a Complex Network through the Betweenn...,ISCAS,1


# Train and validation Split

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label.values, 
                                                  test_size=0.15, 
                                                  random_state=42, 
                                                  stratify=df.label.values)

#print(X_train)
#print(X_val)

df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'
df.head()

df.groupby(['Conference', 'label', 'data_type']).count()
# df.head()

# Init and load tokenizer and model

In [ ]:
# load tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

# load model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

# move model into device
model.to(device)

# Encoding the data

In [ ]:
df.head()

,Title,Conference,label,data_type
0,Innovation in Database Management: Computer Sc...,VLDB,0,train
1,High performance prime field multiplication fo...,ISCAS,1,train
2,enchanted scissors: a scissor interface for su...,SIGGRAPH,2,train
3,Detection of channel degradation attack by Int...,INFOCOM,3,train
4,Pinning a Complex Network through the Betweenn...,ISCAS,1,train


In [ ]:
title_train_df = df[ df['data_type']=='train' ]['Title'].values
label_train_df = df[ df['data_type']=='train' ]['label'].values
encoded_data_train = tokenizer.batch_encode_plus(
    title_train_df, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

title_val_df = df[ df['data_type']=='val' ]['Title'].values
label_val_df = df[ df['data_type']=='val' ]['label'].values
encoded_data_val = tokenizer.batch_encode_plus(
    title_val_df, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)


input_ids_train_pt = encoded_data_train['input_ids']
attention_masks_train_pt = encoded_data_train['attention_mask']
labels_train_pt = torch.tensor(label_train_df)

input_ids_val_pt = encoded_data_val['input_ids']
attention_masks_val_pt = encoded_data_val['attention_mask']
labels_val_pt = torch.tensor(label_val_df)

dataset_train = TensorDataset(input_ids_train_pt, attention_masks_train_pt, labels_train_pt)
dataset_val = TensorDataset(input_ids_val_pt, attention_masks_val_pt, labels_val_pt)

# Define Data Loaders

In [ ]:
batch_size = 3

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

# Define Optimizer and Scheduler

In [ ]:
optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)
                  
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

# Training loop

In [ ]:
for epoch in range(1, epochs+1):
    loss_train_total = 0
    model.train()
    model.zero_grad()

    for batch in dataloader_train:
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        # print({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
                 
    print(f'\n--- epoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    print(f'Training loss: {loss_train_avg}')

print('--- saving models')
# Logic to save the model

# Evaluation / Validation

In [ ]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')
        
def evaluate(dataloader_val):
    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for step, batch in enumerate(dataloader_val):
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
  
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [ ]:
val_loss, predictions, true_vals = evaluate(dataloader_validation)
val_f1 = f1_score_func(predictions, true_vals)
print(f'Validation loss: {val_loss}')
print(f'F1 Score (Weighted): {val_f1}')

Validation loss: 0.8228503850120996
F1 Score (Weighted): 0.8398327561007215


# Predict

In [ ]:
# helper function
def predict(title):
  enc_title = tokenizer.encode_plus(title, return_tensors="pt")['input_ids'].to(device)
  
  logits = model(enc_title)[0]
  probabilities = torch.softmax(logits, dim=1).detach().cpu().tolist()[0]

  proba_vldb = round(probabilities[0], 2) 
  proba_iscas = round(probabilities[1], 2)
  proba_siggraph = round(probabilities[2], 2)
  proba_infocom = round(probabilities[3], 2)
  proba_www = round(probabilities[4], 2)

  # Input
  print(f"title: {title}")
  # Output (classification)
  print(f"proba_vldb: {proba_vldb}")
  print(f"proba_iscas: {proba_iscas}")
  print(f"proba_siggraph: {proba_siggraph}")
  print(f"proba_infocom: {proba_infocom}")
  print(f"proba_www: {proba_www}")

In [ ]:
predict('An Explanation for Unexpected 802.11 Outdoor Link-level Measurement Results')

title: An Explanation for Unexpected 802.11 Outdoor Link-level Measurement Results
proba_vldb: 0.0
proba_iscas: 0.0
proba_siggraph: 0.0
proba_infocom: 1.0
proba_www: 0.0


# Save model

In [ ]:
!mkdir models
!ls models

In [ ]:
torch.save(model.state_dict(), './models/multiclass-model.bin')

In [ ]:
!ls -lah ./models/multiclass-model.bin

-rw-r--r-- 1 root root 418M Jun 10 14:56 ./models/multiclass-model.bin


# Load model

In [ ]:
# load model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.load_state_dict(torch.load('./models/multiclass-model.bin'))
model.eval()

# move model into device
model.to(device)